In [1]:
!pip install catboost
!pip install xgboost
!pip install openml
!pip install numba

In [2]:
#specify GPU to use
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [3]:
from sklearn.model_selection import train_test_split
import openml
import category_encoders as ce
import numpy as np
import sklearn

# Load SpeedDating dataset
dataset = openml.datasets.get_dataset(40536, download_data=True, download_qualities=True, download_features_meta_data=True)
X, y, categorical_indicator, attribute_names = dataset.get_data(target=dataset.default_target_attribute)
categorical_feature_indices = [idx for idx, idx_bool in enumerate(categorical_indicator) if idx_bool]

X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train, X_valid, y_train, y_valid = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42)

print("Training set size:", len(X_train))
print("Validation set size:", len(X_valid))
print("Test set size:", len(X_test))

X_train.head()

Training set size: 5361
Validation set size: 1341
Test set size: 1676


,has_null,wave,gender,age,age_o,d_age,d_d_age,race,race_o,samerace,...,expected_num_interested_in_me,expected_num_matches,d_expected_happy_with_sd_people,d_expected_num_interested_in_me,d_expected_num_matches,like,guess_prob_liked,d_like,d_guess_prob_liked,met
813,0,3,female,23.0,21.0,2,[2-3],European/Caucasian-American,European/Caucasian-American,1,...,15.0,3.0,[0-4],[10-20],[3-5],5.0,10.0,[0-5],[7-10],0.0
1677,1,5,female,20.0,18.0,2,[2-3],Black/African American,Asian/Pacific Islander/Asian-American,0,...,NaN,3.0,[5-6],[0-3],[3-5],7.0,7.0,[6-8],[7-10],0.0
2915,1,9,female,28.0,27.0,1,[0-1],Black/African American,Asian/Pacific Islander/Asian-American,0,...,NaN,0.0,[7-10],[0-3],[0-2],NaN,NaN,[0-5],[0-4],NaN
5972,1,15,female,28.0,24.0,4,[4-6],European/Caucasian-American,European/Caucasian-American,1,...,NaN,1.0,[0-4],[0-3],[0-2],3.0,5.0,[0-5],[5-6],0.0
5443,1,14,male,23.0,24.0,1,[0-1],Asian/Pacific Islander/Asian-American,Other,0,...,NaN,4.0,[7-10],[0-3],[3-5],5.0,4.0,[0-5],[0-4],0.0


In [4]:
from GRANDE import GRANDE

params = {
        'depth': 5, # tree depth
        'n_estimators': 2048, # number of estimators / trees

        'learning_rate_weights': 0.005, # learning rate for leaf weights
        'learning_rate_index': 0.01, # learning rate for split indices
        'learning_rate_values': 0.01, # learning rate for split values
        'learning_rate_leaf': 0.01, # learning rate for leafs (logits)

        'optimizer': 'adam', # optimizer
        'cosine_decay_steps': 0, # decay steps for lr schedule (CosineDecayRestarts)

        'loss': 'crossentropy', # loss function (default 'crossentropy' for binary & multi-class classification and 'mse' for regression)
        'focal_loss': False, # use focal loss {True, False}
        'temperature': 0.0, # temperature for stochastic re-weighted GD (0.0, 1.0)

        'from_logits': True, # use logits for weighting {True, False}
        'use_class_weights': True, # use class weights for training {True, False}

        'dropout': 0.0, # dropout rate (here, dropout randomly disables individual estimators of the ensemble during training)

        'selected_variables': 0.8, # feature subset percentage (0.0, 1.0)
        'data_subset_fraction': 1.0, # data subset percentage (0.0, 1.0)
}

args = {
    'epochs': 1_000, # number of epochs for training
    'early_stopping_epochs': 25, # patience for early stopping (best weights are restored)
    'batch_size': 64,  # batch size for training

    'cat_idx': categorical_feature_indices, # put list of categorical indices
    'objective': 'binary', # objective / task {'binary', 'classification', 'regression'}
    
    'random_seed': 42,
    'verbose': 1,       
}

model_grande = GRANDE(params=params, args=args)

model_grande.fit(X_train=X_train,
          y_train=y_train,
          X_val=X_valid,
          y_val=y_valid)

preds_grande = model_grande.predict(X_test)

2024-03-14 12:06:21.633088: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-14 12:06:25.488848: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-03-14 12:06:30.839657: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2024-03-14 12:06:30.840095: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46090 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:41:00.0, compute capability: 8.6


Epoch 1/1000


I0000 00:00:1710414394.307961  215012 service.cc:145] XLA service 0x7fef88009810 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1710414394.308004  215012 service.cc:153]   StreamExecutor device (0): NVIDIA RTX A6000, Compute Capability 8.6
2024-03-14 12:06:34.374693: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-03-14 12:06:34.663970: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8906


41/84 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6613

I0000 00:00:1710414395.603690  215012 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


84/84 ━━━━━━━━━━━━━━━━━━━━ 6s 32ms/step - loss: 0.6445 - val_loss: 0.5231
Epoch 2/1000
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.4851 - val_loss: 0.4132
Epoch 3/1000
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.3891 - val_loss: 0.3647
Epoch 4/1000
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.3453 - val_loss: 0.3479
Epoch 5/1000
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.3287 - val_loss: 0.3428
Epoch 6/1000
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.3209 - val_loss: 0.3397
Epoch 7/1000
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.3152 - val_loss: 0.3374
Epoch 8/1000
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.3105 - val_loss: 0.3361
Epoch 9/1000
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.3066 - val_loss: 0.3360
Epoch 10/1000
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.3030 - val_loss: 0.3343
Epoch 11/1000
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.2999 - val_loss: 0.3327
Epoch 12/1000
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.2

In [5]:
def calculate_sample_weights(y_data):
    class_weights = sklearn.utils.class_weight.compute_class_weight(class_weight = 'balanced', classes = np.unique(y_data), y = y_data)
    sample_weights = sklearn.utils.class_weight.compute_sample_weight(class_weight = 'balanced', y =y_data)
    return sample_weights

In [6]:
try:
    y_train = y_train.values.codes.astype(np.float64)
    y_valid = y_valid.values.codes.astype(np.float64)
    y_test = y_test.values.codes.astype(np.float64)
except:
    y_train = y_train.values.astype(np.float64)
    y_valid = y_valid.values.astype(np.float64)
    y_test = y_test.values.astype(np.float64)
    
encoder = ce.OrdinalEncoder(cols=X_train.columns[categorical_feature_indices])
encoder.fit(X_train)
X_train = encoder.transform(X_train)
X_valid = encoder.transform(X_valid)
X_test = encoder.transform(X_test)

median = X_train.median(axis=0)
X_train = X_train.fillna(median)
X_valid = X_valid.fillna(median)
X_test = X_test.fillna(median)

X_train_raw = X_train.copy()
X_valid_raw = X_valid.copy()
X_test_raw = X_test.copy()

In [7]:
low_cardinality_indices = []
high_cardinality_indices = []

categorical_feature_indices = []
for column_index in range(X_train.shape[1]):
    if categorical_indicator[column_index]:
        categorical_feature_indices.append(column_index)
        if len(X_train.iloc[:,column_index].unique()) < 5:
            low_cardinality_indices.append(X_train.columns[column_index])
        else:
            high_cardinality_indices.append(X_train.columns[column_index])

In [8]:
encoder = ce.LeaveOneOutEncoder(cols=high_cardinality_indices)
encoder.fit(X_train, y_train)
X_train = encoder.transform(X_train)
X_valid = encoder.transform(X_valid)
X_test = encoder.transform(X_test)

encoder = ce.OneHotEncoder(cols=low_cardinality_indices)
encoder.fit(X_train)
X_train = encoder.transform(X_train)
X_valid = encoder.transform(X_valid)
X_test = encoder.transform(X_test)

median = X_train.median(axis=0)
X_train = X_train.fillna(median)
X_valid = X_valid.fillna(median)
X_test = X_test.fillna(median)

quantile_noise = 1e-4
quantile_train = np.copy(X_train.values).astype(np.float64)
np.random.seed(42)
stds = np.std(quantile_train, axis=0, keepdims=True)
noise_std = quantile_noise / np.maximum(stds, quantile_noise)
quantile_train += noise_std * np.random.randn(*quantile_train.shape)       

scaler = sklearn.preprocessing.QuantileTransformer(output_distribution='normal')
scaler.fit(quantile_train)

X_train = scaler.transform(X_train.values.astype(np.float64))
X_valid = scaler.transform(X_valid.values.astype(np.float64))
X_test = scaler.transform(X_test.values.astype(np.float64))

In [9]:
from xgboost import XGBClassifier
model_xgb = XGBClassifier(n_estimators=1000, early_stopping_rounds=20)
model_xgb.fit(X_train, 
                y_train, 
                sample_weight=calculate_sample_weights(y_train), 
                eval_set=[(X_valid, y_valid)], 
                sample_weight_eval_set=[calculate_sample_weights(y_valid)]
             )
if args['objective'] == 'regression':
    preds_xgb = model_xgb.predict(X_test)
else:
    preds_xgb = model_xgb.predict_proba(X_test)

[0]	validation_0-logloss:0.61630
[1]	validation_0-logloss:0.58654
[2]	validation_0-logloss:0.57133
[3]	validation_0-logloss:0.56570
[4]	validation_0-logloss:0.57115
[5]	validation_0-logloss:0.56865
[6]	validation_0-logloss:0.57523
[7]	validation_0-logloss:0.58349
[8]	validation_0-logloss:0.59104
[9]	validation_0-logloss:0.60149
[10]	validation_0-logloss:0.61173
[11]	validation_0-logloss:0.62385
[12]	validation_0-logloss:0.62838
[13]	validation_0-logloss:0.64028
[14]	validation_0-logloss:0.64788
[15]	validation_0-logloss:0.64897
[16]	validation_0-logloss:0.66066
[17]	validation_0-logloss:0.66165
[18]	validation_0-logloss:0.66946
[19]	validation_0-logloss:0.67140
[20]	validation_0-logloss:0.67064
[21]	validation_0-logloss:0.67684
[22]	validation_0-logloss:0.68804


In [10]:
from catboost import CatBoostClassifier, Pool

model_catboost = CatBoostClassifier(n_estimators=1000, 
                                    early_stopping_rounds=20)
train_data = Pool(
        data=X_train_raw,
        label=y_train,
        cat_features=categorical_feature_indices,
        weight=calculate_sample_weights(y_train)
    )

eval_data = Pool(
        data=X_valid_raw,
        label=y_valid,
        cat_features=categorical_feature_indices,
        weight=calculate_sample_weights(y_valid),
    )

model_catboost.fit(X=train_data, 
                   eval_set=eval_data)

if args['objective'] == 'regression':
    preds_catboost = model_catboost.predict(X_test_raw)
else:
    preds_catboost = model_catboost.predict_proba(X_test_raw)


Learning rate set to 0.047977
0:	learn: 0.6738749	test: 0.6755616	best: 0.6755616 (0)	total: 175ms	remaining: 2m 54s
1:	learn: 0.6590376	test: 0.6621054	best: 0.6621054 (1)	total: 254ms	remaining: 2m 6s
2:	learn: 0.6476492	test: 0.6530350	best: 0.6530350 (2)	total: 312ms	remaining: 1m 43s
3:	learn: 0.6334984	test: 0.6409063	best: 0.6409063 (3)	total: 425ms	remaining: 1m 45s
4:	learn: 0.6211706	test: 0.6295918	best: 0.6295918 (4)	total: 528ms	remaining: 1m 45s
5:	learn: 0.6089204	test: 0.6190270	best: 0.6190270 (5)	total: 628ms	remaining: 1m 44s
6:	learn: 0.5978930	test: 0.6102024	best: 0.6102024 (6)	total: 727ms	remaining: 1m 43s
7:	learn: 0.5884188	test: 0.6025458	best: 0.6025458 (7)	total: 821ms	remaining: 1m 41s
8:	learn: 0.5827542	test: 0.5988541	best: 0.5988541 (8)	total: 862ms	remaining: 1m 34s
9:	learn: 0.5709456	test: 0.5900052	best: 0.5900052 (9)	total: 959ms	remaining: 1m 34s
10:	learn: 0.5629534	test: 0.5835565	best: 0.5835565 (10)	total: 1.07s	remaining: 1m 36s
11:	learn: 0

In [11]:
if args['objective'] == 'binary':
    accuracy = sklearn.metrics.accuracy_score(y_test, np.round(preds_grande[:,1]))
    f1_score = sklearn.metrics.f1_score(y_test, np.round(preds_grande[:,1]), average='macro')
    roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_grande[:,1], average='macro', multi_class='ovo')

    print('Accuracy GRANDE:', accuracy)
    print('F1 Score GRANDE:', f1_score)
    print('ROC AUC GRANDE:', roc_auc)
    print('\n')

    accuracy = sklearn.metrics.accuracy_score(y_test, np.round(preds_xgb[:,1]))
    f1_score = sklearn.metrics.f1_score(y_test, np.round(preds_xgb[:,1]), average='macro')
    roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_xgb[:,1], average='macro', multi_class='ovo')

    print('Accuracy XGB:', accuracy)
    print('F1 Score XGB:', f1_score)
    print('ROC AUC XGB:', roc_auc)
    print('\n')

    accuracy = sklearn.metrics.accuracy_score(y_test, np.round(preds_catboost[:,1]))
    f1_score = sklearn.metrics.f1_score(y_test, np.round(preds_catboost[:,1]), average='macro')
    roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_catboost[:,1], average='macro', multi_class='ovo')

    print('Accuracy CatBoost:', accuracy)
    print('F1 Score CatBoost:', f1_score)
    print('ROC AUC CatBoost:', roc_auc)
    print('\n')
elif args['objective'] == 'classification':
    accuracy = sklearn.metrics.accuracy_score(y_test, np.argmax(preds_grande, axis=1))
    f1_score = sklearn.metrics.f1_score(y_test, np.argmax(preds_grande, axis=1), average='macro')
    roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_grande, average='macro', multi_class='ovo', labels=[i for i in range(preds_grande.shape[1])])

    print('Accuracy GRANDE:', accuracy)
    print('F1 Score GRANDE:', f1_score)
    print('ROC AUC GRANDE:', roc_auc)
    print('\n')

    accuracy = sklearn.metrics.accuracy_score(y_test, np.argmax(preds_xgb, axis=1))
    f1_score = sklearn.metrics.f1_score(y_test, np.argmax(preds_xgb, axis=1), average='macro')
    roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_xgb, average='macro', multi_class='ovo', labels=[i for i in range(preds_grande.shape[1])])

    print('Accuracy XGB:', accuracy)
    print('F1 Score XGB:', f1_score)
    print('ROC AUC XGB:', roc_auc)
    print('\n')

    accuracy = sklearn.metrics.accuracy_score(y_test, np.argmax(preds_catboost, axis=1))
    f1_score = sklearn.metrics.f1_score(y_test, np.argmax(preds_catboost, axis=1), average='macro')
    roc_auc = sklearn.metrics.roc_auc_score(y_test, preds_catboost, average='macro', multi_class='ovo', labels=[i for i in range(preds_grande.shape[1])])

    print('Accuracy CatBoost:', accuracy)
    print('F1 Score CatBoost:', f1_score)
    print('ROC AUC CatBoost:', roc_auc)
    print('\n')
else:
    mean_absolute_error = sklearn.metrics.mean_absolute_error(y_test, np.round(preds_grande))
    r2_score = sklearn.metrics.r2_score(y_test, np.round(preds_grande))

    print('MAE GRANDE:', mean_absolute_error)
    print('R2 Score GRANDE:', r2_score)
    print('\n')

    mean_absolute_error = sklearn.metrics.mean_absolute_error(y_test, np.round(preds_xgb))
    r2_score = sklearn.metrics.r2_score(y_test, np.round(preds_xgb))

    print('MAE XGB:', mean_absolute_error)
    print('R2 Score XGB:', r2_score)
    print('\n')

    mean_absolute_error = sklearn.metrics.mean_absolute_error(y_test, np.round(preds_catboost))
    r2_score = sklearn.metrics.r2_score(y_test, np.round(preds_catboost))

    print('MAE CatBoost:', mean_absolute_error)
    print('R2 Score CatBoost:', r2_score)
    print('\n')

Accuracy GRANDE: 0.8526252983293556
F1 Score GRANDE: 0.7202073309285757
ROC AUC GRANDE: 0.8639484451000019


Accuracy XGB: 0.7655131264916468
F1 Score XGB: 0.6770993118601543
ROC AUC XGB: 0.7998723168348623


Accuracy CatBoost: 0.7869928400954654
F1 Score CatBoost: 0.7046369469890426
ROC AUC CatBoost: 0.8625913411247657




In [12]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()